In [1]:
from pynq import Overlay
import numpy as np
import random
from pynq import allocate

In [2]:
overlay = Overlay('aes.bit')
overlay?

In [3]:
aes_ip = overlay.aes_0
dma = overlay.axi_dma
dma_send = overlay.axi_dma.sendchannel
dma_recv = overlay.axi_dma.recvchannel

In [4]:
aes_mode = 256

if aes_mode == 128:
    cipherkey_size = 16
    key_array = [ord(char) for char in "kkkkeeeeyyyy...."]
    aes_ip.write(0x10, 128) # Write Mode (128 for AES-128, 192 for AES-192, 256 for AES-256)
    
elif aes_mode == 192:
    cipherkey_size = 24
    key_array = [ord(char) for char in "kkkkeeeeyyyy....12345678"]
    aes_ip.write(0x10, 192) # Write Mode (128 for AES-128, 192 for AES-192, 256 for AES-256)
    
elif aes_mode == 256:
    cipherkey_size = 32
    key_array = [ord(char) for char in "kkkkeeeeyyyy....12345678ABCDEFGH"]
    aes_ip.write(0x10, 256) # Write Mode (128 for AES-128, 192 for AES-192, 256 for AES-256)
    
text_size = 16
plaintext_array = [ord(char) for char in "abcdef1234567890"]

key_and_plaintext_buffer = allocate(shape=(cipherkey_size + text_size,), dtype=np.uint8)
key_and_plaintext_buffer[:] = key_array + plaintext_array
ciphertext_and_decryptedtext_buffer = allocate(shape=(2*text_size,), dtype=np.uint8)

In [5]:
CONTROL_REGISTER = 0x0
aes_ip.write(CONTROL_REGISTER, 0x81) # Set AP_START and AUTO_RESTART to 1
aes_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  mode = Register(mode=write-only)
}

In [6]:
def run_kernel():
    dma_send.transfer(key_and_plaintext_buffer)
    dma_recv.transfer(ciphertext_and_decryptedtext_buffer)

In [7]:
run_kernel()

In [8]:
# Print out plaintext portion of buffer
print("Plaintext (hex): ")
for i in range(16):
    print(format(key_and_plaintext_buffer[cipherkey_size + i], '02x'), end=' ')

Plaintext (hex): 
61 62 63 64 65 66 31 32 33 34 35 36 37 38 39 30 

In [9]:
# Print out ciphertext_buffer
print("Ciphertext (hex): ")
for i in range(16):
    print(format(ciphertext_and_decryptedtext_buffer[i], '02x'), end=' ')

Ciphertext (hex): 
32 cd ba 88 5c e3 af 76 7a 38 7e bb 54 84 62 b5 

In [10]:
# Print out decryptedtext_buffer
print("Decryptedtext (hex): ")
for i in range(16):
    print(format(ciphertext_and_decryptedtext_buffer[i+16], '02x'), end=' ')

Decryptedtext (hex): 
61 62 63 64 65 66 31 32 33 34 35 36 37 38 39 30 

In [11]:
key_and_plaintext_buffer

PynqBuffer([107, 107, 107, 107, 101, 101, 101, 101, 121, 121, 121, 121,
             46,  46,  46,  46,  49,  50,  51,  52,  53,  54,  55,  56,
             65,  66,  67,  68,  69,  70,  71,  72,  97,  98,  99, 100,
            101, 102,  49,  50,  51,  52,  53,  54,  55,  56,  57,  48],
           dtype=uint8)

In [12]:
ciphertext_and_decryptedtext_buffer

PynqBuffer([ 50, 205, 186, 136,  92, 227, 175, 118, 122,  56, 126, 187,
             84, 132,  98, 181,  97,  98,  99, 100, 101, 102,  49,  50,
             51,  52,  53,  54,  55,  56,  57,  48], dtype=uint8)

In [13]:
del key_and_plaintext_buffer
del ciphertext_and_decryptedtext_buffer